In [1]:
with open("schema.txt", "r") as file:
    schema = file.read()
print(schema)

CREATE TABLE Users (
    user_id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(100) NOT NULL,
    email VARCHAR(100) UNIQUE NOT NULL,
    phone VARCHAR(15),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE Products (
    product_id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(150) NOT NULL,
    description TEXT,
    price DECIMAL(10,2) NOT NULL,
    stock INT DEFAULT 0,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE Orders (
    order_id INT PRIMARY KEY AUTO_INCREMENT,
    user_id INT NOT NULL,
    order_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    total_amount DECIMAL(10,2) NOT NULL,
    status VARCHAR(20) DEFAULT 'Pending',
    FOREIGN KEY (user_id) REFERENCES Users(user_id)
);

CREATE TABLE OrderItems (
    order_item_id INT PRIMARY KEY AUTO_INCREMENT,
    order_id INT NOT NULL,
    product_id INT NOT NULL,
    quantity INT NOT NULL,
    price DECIMAL(10,2) NOT NULL,
    FOREIGN KEY (order_id) REFERENCES Orders(order_id),
    FOREIGN K

In [2]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.2:latest", temperature=0.1, max_tokens=512)


In [ ]:
from langchain.prompts import PromptTemplate

template = PromptTemplate(
    input_variables=["schema", "dialect", "input"],
    template="""You are a SQL expert. 
    Generate a valid {dialect} SQL query for the following request:
    Question: {input}

    Schema:
    {schema}

    Rules:
    - Output ONLY the SQL query (no explanation, no comments, no extra text).
    - Use explicit column names (never use SELECT *).
    - Ensure the query is syntactically correct for {dialect}.
    - Do not include markdown formatting, quotes, or backticks.

    Final Output:
    """
)

In [4]:
prompt = template.format(dialect="MySQL",schema=schema,input="how many user do we have?")
result = llm.invoke(prompt)
print(result.content)

SELECT COUNT(DISTINCT user_id) FROM Users;


In [5]:
prompt = template.format(dialect="MySQL",schema=schema, input="List all orders with the customer’s name?")
result = llm.invoke(prompt)
print(result.content)

SELECT o.user_id, u.name FROM Orders o JOIN Users u ON o.user_id = u.user_id


In [6]:
from pprint import pprint
prompt = template.format(dialect="MySQL",
                         schema=schema, 
                         input="Find all products purchased in a specific order?.")
result = llm.invoke(prompt)
pprint(result.content)

('SELECT o.order_id, p.name, oi.quantity \n'
 'FROM Orders o \n'
 'JOIN OrderItems oi ON o.order_id = oi.order_id \n'
 'JOIN Products p ON oi.product_id = p.product_id \n'
 'WHERE o.user_id = ?;')


In [7]:
prompt = template.format(dialect="MySQL",schema=schema, input="Get all users who purchased more than 3 products in a single order.")
result = llm.invoke(prompt)
print(result.content)

SELECT u.name, COUNT(o.order_id) AS num_orders 
FROM Users u
JOIN Orders o ON u.user_id = o.user_id
GROUP BY u.user_id
HAVING COUNT(DISTINCT o.order_id) > 3


In [8]:
prompt = template.format(dialect="MySQL",schema=schema, input="Find the average order value for each user.")
result = llm.invoke(prompt)
print(result.content)

SELECT AVG(oi.price) AS average_order_value FROM OrderItems oi JOIN Orders o ON oi.order_id = o.order_id JOIN Users u ON o.user_id = u.user_id GROUP BY u.user_id


In [11]:
prompt = template.format(dialect="MySQL",schema=schema, input="Show orders with more than one payment.")
result = llm.invoke(prompt)
print(result.content)

SELECT o.order_id, o.user_id, COUNT(DISTINCT p.payment_id) as num_payments
FROM Orders o
JOIN Payments p ON o.order_id = p.order_id
GROUP BY o.order_id, o.user_id
HAVING COUNT(DISTINCT p.payment_id) > 1


In [12]:
prompt = template.format(dialect="MySQL",schema=schema, input="Get all products along with the number of unique customers who bought them..")
result = llm.invoke(prompt)
print(result.content)

SELECT p.name, COUNT(DISTINCT u.user_id) FROM Products p JOIN OrderItems oi ON p.product_id = oi.product_id JOIN Orders o ON oi.order_id = o.order_id JOIN Users u ON o.user_id = u.user_id GROUP BY p.name


In [13]:
prompt = template.format(dialect="MySQL",schema=schema, input="Find the total revenue generated per product.")
result = llm.invoke(prompt)
print(result.content)

SELECT p.name, SUM(oi.quantity * oi.price) AS total_revenue FROM Products p JOIN OrderItems oi ON p.product_id = oi.product_id GROUP BY p.name


In [10]:
!ollama list

NAME                   ID              SIZE      MODIFIED     
mistral:7b             6577803aa9a0    4.4 GB    17 hours ago    
granite3.1-dense:2b    fba1ad01113e    1.6 GB    2 weeks ago     
gemma3:270m            e7d36fb2c3b3    291 MB    2 weeks ago     
llama3.2:latest        a80c4f17acd5    2.0 GB    3 weeks ago     
